In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from data_loader import *

2023-08-25 19:29:42.968293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-25 19:29:43.965468: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
train_df, test_df = load_indoml_data('./data/public_dat')

loading test data: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 103683/103683 [00:00<00:00, 106631.40it/s]


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text  # Needed for loading universal-sentence-encoder-cmlm/multilingual-preprocess
import numpy as np

preprocessor = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer(
    "https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base-br/1", trainable=False)

# english_embeds = encoder(preprocessor(english_sentences))["default"]
# japanese_embeds = encoder(preprocessor(japanese_sentences))["default"]
# italian_embeds = encoder(preprocessor(italian_sentences))["default"]

2023-08-25 19:35:32.780339: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-08-25 19:35:32.781098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-08-25 19:35:32.781247: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-08-25 19:35:32.781821: I tensorflow/core/

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from pandas import Series

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
x = preprocessor(text_input)
x = encoder(x)["default"]
x = layers.Dense(train_labels.shape[-1], activation='softmax')(x)

model = Model(inputs=text_input, outputs=x)
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

In [ ]:
model.load_weights("./checkpoints/models_weights.keras")